# Dataset sockshop (9n6mf)

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
DATASET_ID = "9n6mf"

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [9]:
import sys
sys.path.append('../../')

from tsdr import tsdr
from eval import groundtruth
from meltria import loader
from eval import validation

In [6]:
metrics_files = !find "/datasets/argowf-chaos-{DATASET_ID}/" -type f -name "*.json"
dataset_generator = loader.load_dataset_as_generator(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=4*45, # 45min
)
records = [r for rec in dataset_generator for r in rec]

In [7]:
len(records)

73

In [11]:
well_injected_records = validation.find_records_detected_anomalies_of_sli(records, faulty_datapoints=4*5, sli_index=1)

In [12]:
len(well_injected_records)

50

In [15]:
well_injected_records = validation.find_records_detected_anomalies_of_cause_metrics(well_injected_records, faulty_datapoints=4*5)

In [17]:
len(well_injected_records)

50

In [18]:
import sys
sys.path.append("../")
from notebooklib.save import run_tsdr, save_tsdr, load_tsdr

In [19]:
for items in run_tsdr(well_injected_records):
    save_tsdr(DATASET_ID, *items)

In [ ]:
for items in run_tsdr(records, tsdr_options={
    "step2_clustering_method_name": "dbscan",
    "step2_dbscan_min_pts": 1,
    "step2_dbscan_dist_type": "pearsonr",  # 'pearsonr' or 'sbd'
    "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
    "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
    "step2_clustering_choice_method": "max_cluster",  # 'medoid' or 'maxsum'
}):
    save_tsdr(DATASET_ID, *items, suffix="fluxrank_max_cluster")